In [ ]:
import os
import glob
import imageio
import numpy as np
import cv2

In [ ]:
from matplotlib import pyplot as plt
from skimage.transform import resize
from hog import hog


files = glob.glob("ethz-cil-road-segmentation-2023/training/images/*.png") + glob.glob("ethz-cil-road-segmentation-2023/test/images/*.png")
#imgs = [np.concatenate([imageio.imread(file)[:,:,:3],imageio.imread(file.replace("images","groundtruth"))[:,:,None]], axis=-1) for file in files]
original_imgs = [imageio.imread(file) for file in files]
hogs = [hog(cv2.cvtColor((img*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)) for img in original_imgs]
imgs = [resize(image, (image.shape[0] // 4, image.shape[1] // 4),
                       anti_aliasing=True) for image in original_imgs]
plt.imshow(imgs[np.random.randint(0, len(files))])

print(imgs[0].shape)

In [ ]:
def left_trace(img):
    return img[:,0]#.flatten()
def right_trace(img):
    return img[:,-1]#.flatten()
def upper_trace(img):
    return img[-1,:]#.flatten()
def lower_trace(img):
    return img[0,:]#.flatten()

In [ ]:
ix = np.random.randint(0, len(files))
plt.imshow(imgs[ix])
plt.show()
plt.imshow(left_trace(imgs[ix]))
plt.show()
print(imgs[ix].shape)
gray = cv2.cvtColor((imgs[ix]*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
plt.imshow(gray)
plt.show()
feat = hog(gray)

In [ ]:
import skimage
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pca = PCA(100) # we need 2 principal components.
converted_data = np.reshape([np.mean(img, axis=(0,1)) for img in imgs], (len(imgs),-1))
clusters = KMeans(2).fit_transform(converted_data)

In [ ]:
_imgs = np.array(imgs) #[np.argmax(clusters, axis=1) == 0]
print(len(imgs), len(_imgs))

In [ ]:
left_color = np.array([left_trace(img) for img in _imgs])
right_color = np.array([right_trace(img) for img in _imgs])
upper_color = np.array([upper_trace(img) for img in _imgs])
lower_color = np.array([lower_trace(img) for img in _imgs])
# left_hog = np.array([left_trace(img) for img in hogs])
# right_hog = np.array([right_trace(img) for img in hogs])
# upper_hog = np.array([upper_trace(img) for img in hogs])
# lower_hog = np.array([lower_trace(img) for img in hogs])

print(left_color.shape)

print((left_color[None, :] - right_color[:, None]).shape)
# print((left_hog[None, :] - right_hog[:, None]).shape)

def get_sim(traces1, traces2):
    maes = [np.mean(np.linalg.norm(traces1[None, :] - traces2[:, None], axis=-1), axis=-1)]
    for i in range(1,6):
        maes.append(np.mean(np.linalg.norm(traces1[None, :, i:] - traces2[:, None, :-i], axis=-1), axis=-1))
        maes.append(np.mean(np.linalg.norm(traces1[None, :, :-i] - traces2[:, None, i:], axis=-1), axis=-1))
    return np.min(maes, axis=0)

lr_similarities_color = get_sim(left_color, right_color)
ul_similarities_color = get_sim(upper_color, lower_color)
# lr_similarities_hog = np.mean(np.linalg.norm(left_hog[None, :] - right_hog[:, None], axis=-1), axis=-1)
# ul_similarities_hog = np.mean(np.linalg.norm(upper_hog[None, :] - lower_hog[:, None], axis=-1), axis=-1)

ul_similarities_color /= ul_similarities_color.sum(axis=0)[None:]
lr_similarities_color /= lr_similarities_color.sum(axis=0)[None:]
# ul_similarities_hog /= ul_similarities_hog.sum(axis=0)[None:]
# lr_similarities_hog /= lr_similarities_hog.sum(axis=0)[None:]

lr_similarities =  lr_similarities_color
ul_similarities =  ul_similarities_color
plt.matshow(lr_similarities_color)
plt.show()
#plt.matshow(lr_similarities_hog)
# plt.show()

In [ ]:
best = np.argsort(lr_similarities.flatten())
for b in best[:10]:
    i1, i2 = np.unravel_index(b, ul_similarities.shape)
    joint_img = np.concatenate([imgs[i1],imgs[i2]], axis=1)
    plt.imshow(joint_img)
    plt.title(str(lr_similarities[i1,i2]))
    plt.show()


In [ ]:
quadruple_costs = lr_similarities[:,:,None, None] + lr_similarities[None,None,:,:] + ul_similarities[:,None,:,None] + ul_similarities[None,:,None,:]
# quadruple_costs /= quadruple_costs.sum(axis=0)[None]
print(quadruple_costs.shape)

In [ ]:
i1, i2, i3, i4 = np.unravel_index(quadruple_costs.argmin(), quadruple_costs.shape)
print(quadruple_costs[i1,i2,i3,i4])
joint_img = np.concatenate([np.concatenate([imgs[i3],imgs[i4]], axis=1), np.concatenate([imgs[i1],imgs[i2]], axis=1)], axis=0)
plt.imshow(joint_img)
plt.show()

In [ ]:
plt.hist(quadruple_costs.flatten())

In [ ]:
best = np.argsort(quadruple_costs.flatten())

In [ ]:
for b in best[:5]:
    i1, i2, i3, i4 = np.unravel_index(b, quadruple_costs.shape)
    joint_img = np.concatenate([np.concatenate([imgs[i3],imgs[i4]], axis=1), np.concatenate([imgs[i1],imgs[i2]], axis=1)], axis=0)
    plt.imshow(joint_img[:,:,:3])
    plt.show()

In [ ]:

import time

known_quadrupels = [(44, 71, 33, 101), (113, 48, 50, 4), (89, 81, 129, 84), (96, 133, 38, 102), (33, 101, 47, 32), (121, 72, 96, 133), (57, 137, 98, 6), (54, 94, 0, 108), (16, 54, 66, 0), (74, 140, 138, 39), (58, 20, 37, 82), (104, 117, 121, 72), (91, 109, 93, 124), (65, 135, 2, 95), (6, 115, 16, 54), (83, 127, 57, 137), (14, 132, 31, 128)]

print(len(np.array(known_quadrupels).flatten()))
for b in best[:1]:
    i1, i2, i3, i4 = np.unravel_index(b, quadruple_costs.shape)
    if len(set([i1,i2,i3,i4])) != 4 or 53 in [i1,i2,i3,i4] or 131 in [i1,i2,i3,i4] or (i1,i2,i3,i4) in known_quadrupels or i1 in np.array(known_quadrupels).flatten() or i2 in np.array(known_quadrupels).flatten() or i3 in np.array(known_quadrupels).flatten() or i4 in np.array(known_quadrupels).flatten():
        continue
    joint_img = np.concatenate([np.concatenate([imgs[i3],imgs[i4]], axis=1), np.concatenate([imgs[i1],imgs[i2]], axis=1)], axis=0)
    plt.imshow(joint_img[:,:,:3])
    plt.show()
    print(i1,i2,i3,i4)
    time.sleep(1)
    i = input()
    if i == "y":
        known_quadrupels.append((i1,i2,i3,i4))
    elif i == "p":
        time.sleep(5)
    elif i == "x":
        break

In [ ]:
known_lr_pairs = [] #[(i1,i2) for (i1,i2,i3,i4) in known_quadrupels] + [(i3,i4) for (i1,i2,i3,i4) in known_quadrupels]
known_lr_pairs +=[(44, 71), (113, 48), (89, 81), (96, 133), (33, 101), (57, 137), (54, 94), (16, 54), (74, 140), (91, 109), (65, 135), (6, 115), (83, 127), (14, 132), (33, 101), (50, 4), (129, 84), (38, 102), (47, 32), (96, 133), (98, 6), (0, 108), (66, 0), (138, 39), (93, 124), (2, 95), (16, 54), (57, 137), (31, 128), (72, 79), (114, 60), (53, 3), (85, 28), (1, 68), (25, 88), (112, 59), (123, 90), (84, 45), (20, 7), (22, 122), (30, 89), (107, 111), (106, 143), (64, 23), (95, 116), (68, 22), (79, 52), (23, 141), (78, 34), (51, 10), (17, 50), (132, 66), (36, 136), (59, 44), (122, 97), (24, 56), (75, 17), (105, 55), (102, 156), (131, 161), (205, 134), (219, 67), (242, 194), (145, 165), (282, 231), (69, 179), (203, 93), (270, 148), (88, 270), (254, 18), (230, 96), (185, 242), (279, 167), (144, 209), (5, 217), (133, 237), (201, 38), (202, 98), (155, 244), (221, 62), (3, 166), (232, 238), (209, 26), (208, 131), (81, 184), (231, 61), (250, 53), (194, 205), (18, 160), (156, 159), (60, 185), (216, 70), (110, 153), (192, 207), (157, 105), (187, 203), (162, 9), (274, 12), (252, 212), (283, 241), (188, 91), (136, 223), (214, 49), (169, 272), (240, 155), (10, 221), (15, 146), (28, 236), (32, 201), (271, 174), (158, 254), (161, 35), (245, 78), (40, 248), (258, 150), (211, 172), (160, 63), (116, 169), (166, 43), (261, 107), (255, 261), (228, 211), (164, 57), (48, 281), (264, 33), (244, 103), (119, 228), (165, 120), (198, 267), (248, 113), (186, 245), (281, 257), (67, 252), (287, 282), (11, 189), (8, 232), (120, 186), (251, 112), (247, 15), (182, 114), (73, 278), (55, 183), (173, 210), (266, 24), (196, 280), (39, 247), (153, 123), (193, 36), (125, 273), (212, 106), (207, 31), (246, 42), (199, 178), (127, 234), (190, 181), (286, 199), (92, 235), (77, 268), (234, 263), (226, 21), (130, 285), (42, 154), (118, 284), (70, 233), (235, 226), (249, 16), (154, 142), (191, 27), (27, 175), (220, 190), (109, 227), (124, 180), (225, 46), (176, 170), (213, 139), (35, 147), (43, 275), (265, 176), (76, 265), (82, 149), (222, 121), (170, 58), (139, 11), (262, 64), (204, 279)]

print(len(np.array(known_lr_pairs).flatten()))
known_ul_pairs = [] # [(i1,i3) for (i1,i2,i3,i4) in known_quadrupels] + [(i2,i4) for (i1,i2,i3,i4) in known_quadrupels]
known_ul_pairs += [(44, 33), (113, 50), (89, 129), (96, 38), (33, 47), (57, 98), (54, 0), (16, 66), (74, 138), (58, 37), (104, 121), (91, 93), (65, 2), (6, 16), (83, 57), (14, 31), (71, 101), (48, 4), (81, 84), (133, 102), (101, 32), (72, 133), (137, 6), (94, 108), (54, 0), (140, 39), (20, 82), (117, 72), (109, 124), (135, 95), (115, 54), (127, 137), (132, 128), (53, 131), (25, 69), (26, 8), (9, 134), (40, 75), (15, 59), (106, 142), (63, 12), (23, 73), (86, 110), (49, 125), (45, 103), (51, 60), (92, 143), (100, 122), (22, 118), (85, 119), (24, 136), (77, 78), (13, 80), (0, 70), (35, 58), (69, 87), (38, 10), (8, 1), (29, 52), (5, 71), (2, 19), (103, 76), (80, 25), (119, 74), (97, 14), (282, 53), (216, 241), (168, 9), (176, 271), (155, 28), (231, 3), (224, 232), (145, 105), (160, 274), (270, 158), (163, 214), (10, 185), (188, 203), (156, 62), (221, 242), (158, 209), (241, 91), (7, 149), (209, 181), (228, 140), (3, 161), (88, 179), (215, 94), (184, 45), (278, 167), (39, 251), (197, 247), (73, 279), (102, 221), (125, 248), (84, 244), (21, 178), (28, 228), (162, 205), (68, 272), (254, 26), (222, 230), (237, 156), (59, 264), (193, 145), (52, 259), (141, 278), (41, 157), (142, 195), (244, 236), (76, 172), (32, 257), (259, 159), (166, 35), (277, 218), (110, 146), (279, 200), (173, 11), (62, 194), (240, 85), (230, 201), (260, 15), (18, 224), (253, 215), (138, 163), (149, 139), (179, 144), (239, 193), (123, 217), (177, 240), (167, 99), (61, 166), (280, 267), (251, 49), (262, 43), (147, 20), (36, 165), (271, 153), (70, 225), (274, 238), (236, 211), (170, 174), (219, 148), (257, 114), (129, 155), (192, 171), (111, 263), (47, 281), (152, 48), (245, 150), (78, 269), (229, 40), (159, 162), (217, 222), (191, 252), (223, 186), (206, 41), (165, 55), (276, 273), (287, 250), (248, 17), (218, 126), (66, 216), (285, 67), (211, 197), (186, 258), (264, 152), (198, 88), (210, 189), (267, 270), (164, 202), (79, 237), (151, 115), (19, 206), (189, 277), (1, 169), (256, 79), (238, 22), (120, 183), (243, 116), (131, 176), (204, 7), (121, 96)]

# if errors:
#     length = len(known_lr_pairs)
#     known_lr_pairs = [(x,y) for (x,y) in known_lr_pairs if (x not in errors and y not in errors) or (x in errors and y in errors)]
#     print(f"filtered {length - len(known_lr_pairs)} pairs" )
#     length = len(known_ul_pairs)
#     known_ul_pairs = [(x,y) for (x,y) in known_ul_pairs if (x not in errors and y not in errors) or (x in errors and y in errors)]
#     print(f"filtered {length - len(known_ul_pairs)} pairs" )

import networkx as nx
G = nx.Graph()
G.add_edges_from(known_ul_pairs)
G.add_edges_from(known_lr_pairs)
print(G)
nx.draw_networkx(G)

def get_component_map(G):
    components = nx.connected_components(G)
    component_map = { }
    for i, nodes in enumerate(components):
        for node in nodes:
            component_map[int(node)] = i
    return component_map


if False:
    print(len(set(np.array(known_ul_pairs + known_lr_pairs).flatten())))
    best_ul = np.argsort(ul_similarities.flatten())
    for b in best_ul[:10000]:
        component_map = get_component_map(G)
        i1, i2 = np.unravel_index(b, ul_similarities.shape)
        if (i1 in component_map and i2 in component_map and component_map[i1] == component_map[i2]) or len(set([i1,i2])) != 2 or (i1,i2) in known_ul_pairs: # or i1 in np.array(known_ul_pairs).flatten() or i2 in np.array(known_ul_pairs).flatten():
            continue
        joint_img = np.concatenate([original_imgs[i2],original_imgs[i1]], axis=0)
        plt.imshow(joint_img[:,:,:3])
        plt.show()
        print(i1,i2)
        time.sleep(1)
        i = input()
        if i == "y":
            known_ul_pairs.append((i1,i2))
            G.add_edges_from([(i1, i2)])
        elif i == "p":
            time.sleep(5)
        elif i == "x":
            break
else:
    print(len(set(np.array(known_ul_pairs + known_lr_pairs).flatten())))
    best_lr = np.argsort(lr_similarities.flatten())
    for b in best_lr[:10000]:
        component_map = get_component_map(G)
        i1, i2 = np.unravel_index(b, lr_similarities.shape)
        if (i1 in component_map and i2 in component_map and component_map[i1] == component_map[i2]) or len(set([i1,i2])) != 2 or (i1,i2) in known_lr_pairs: # or i1 in np.array(known_ul_pairs).flatten() or i2 in np.array(known_ul_pairs).flatten():
            continue
        joint_img = np.concatenate([original_imgs[i1],original_imgs[i2]], axis=1)
        plt.imshow(joint_img[:,:,:3])
        plt.show()
        print(i1,i2)
        time.sleep(1)
        i = input()
        if i == "y":
            known_lr_pairs.append((i1,i2))
            G.add_edges_from([(i1, i2)])
        elif i == "p":
            time.sleep(5)
        elif i == "x":
            break

In [ ]:
print(known_ul_pairs)

In [ ]:
import networkx as nx
nx.draw_networkx(G)

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_edges_from(known_ul_pairs)
G.add_edges_from(known_lr_pairs)

indices = {}
queue = [(179, (0,0))]
errors = []
for img, (x,y) in queue:
    if img in indices:
        if indices[img] != (x,y):
            print(f"{img} already has index {indices[img]} cannot assign {(x,y)}")
            errors.append(img)
    else:
        indices[img] = (x,y)
        for img2 in range(len(imgs)):
            if img2 not in indices:
                if (img2, img) in known_lr_pairs:
                    queue.append((img2, (x, y-1)))
                if (img, img2) in known_lr_pairs:
                    queue.append((img2, (x, y+1)))
                if (img, img2) in known_ul_pairs:
                    queue.append((img2, (x-1, y)))
                if (img2, img) in known_ul_pairs:
                    queue.append((img2, (x+1, y)))

In [ ]:
errors = [208, 131, 161,  35, 147, 126,
 250, 121,   3, 166,  43, 218,
 287, 282, 231,  61, 262, 277,
 82, 149, 213, 139,  11, 189,
          20, 37,  173, 210, 7, 58, 275, 104, 53]

In [ ]:
print(indices)
_indices = np.array(list(indices.values()))
print(_indices)
x_min, y_min = np.min(_indices, axis=0)
size = np.max(_indices, axis=0)-np.min(_indices, axis=0) + 1
print(size)
full_ixes = np.ones(size) * -1

for k in indices:
    (x,y) = indices[k]
    full_ixes[x-x_min, y-y_min] = k
    # plt.title(str(k) + ": " +str((x-x_min, y-y_min)))
    # plt.subplot(*size, (x-x_min)*size[1] + y-y_min + 1)
    # plt.imshow(imgs[k])
# plt.tight_layout()
# plt.show()
#
# print(size*imgs[0].shape[:2])
full_shape = tuple(size*original_imgs[0].shape[:2]) + (original_imgs[0].shape[-1],)
full_img = np.zeros(full_shape)
print(full_img.shape)
img_h, img_w = original_imgs[0].shape[:2]
for k in indices:
    (x,y) = indices[k]
    ix, iy = (x-x_min)*img_h, (y-y_min)*img_w
    full_img[ix:ix+img_h, iy:iy+img_w] = original_imgs[k]
print(full_img[0:10])
full_img /= 255
print(np.min(full_img), np.max(full_img))
print(full_ixes)
plt.figure(figsize=(50,50))
plt.imshow(full_img)
imageio.imwrite("img3.png", full_img)

In [ ]:
np.savetxt("img2.csv", full_ixes, delimiter=",")

In [ ]:
print(full_ixes)

In [ ]:
plt.imshow(imgs[71])